In [ ]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 863.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import pandas as pd

data = pd.read_parquet("/content/dataset_qa.parquet")


In [ ]:
# Create a function to format the data into the required format
def format_data(input_text, output_text):
    return f"<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে একটি নির্দেশ তৈরি করুন যা এলএলএম দ্বারা প্রতিক্রিয়া উত্পাদনের জন্য ব্যবহৃত হতে পারে.\n\n### input:\n{input_text}\n\n### output:\n{output_text}</s>"

In [ ]:
# Apply the function to each row of the dataset to create the new target column
data['target'] = data.apply(lambda row: format_data(row['instruction'], row['output']), axis=1)

In [ ]:
data['target'][0]


'<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে একটি নির্দেশ তৈরি করুন যা এলএলএম দ্বারা প্রতিক্রিয়া উত্পাদনের জন্য ব্যবহৃত হতে পারে.\n\n### input:\nসুস্থ থাকার তিনটি উপায় বলুন।\n\n### output:\n১. ভারসাম্যপূর্ণ ও পুষ্টিকর খাদ্য খান: আপনার খাবারগুলিতে বিভিন্ন ফলমূল ও সবজি, পাতলা প্রোটিন, পূর্ণ শস্য এবং স্বাস্থ্যকর চর্বি অন্তর্ভুক্ত রয়েছে তা নিশ্চিত করুন। এটি আপনার শরীরকে তার সর্বোত্তম কাজ করার জন্য প্রয়োজনীয় পুষ্টি সরবরাহ করতে সহায়তা করে এবং দীর্ঘস্থায়ী রোগ প্রতিরোধে সহায়তা করতে পারে। ২. নিয়মিত শারীরিক ক্রিয়াকলাপে অংশ নিনঃ হাড়, পেশী এবং হৃদরোগের স্বাস্থ্যকে শক্তিশালী রাখতে ব্যায়াম অত্যন্ত গুরুত্বপূর্ণ। প্রতি সপ্তাহে কমপক্ষে ১৫০ মিনিট মাঝারি বায়ুচলাচল বা ৭৫ মিনিট তীব্র ব্যায়াম করার লক্ষ্য রাখুন। ৩. পর্যাপ্ত ঘুম পানঃ পর্যাপ্ত মানের ঘুম পাওয়া শারীরিক ও মানসিক সুস্থতার জন্য অত্যন্ত গুরুত্বপূর্ণ। এটি মেজাজ নিয়ন্ত্রণে সহায়তা করে, জ্ঞানীয় কার্যকারিতা উন্নত করে এবং স্বাস্থ্যকর বৃদ্ধি এবং রোগ প্রতিরোধ ক্ষমতাকে সমর্থন করে। প্রতি রাতে ৭-৯ ঘন্টা ঘুমের লক্ষ্য রাখুন।</s>'

In [ ]:
# Assuming 'target' is the name of the column containing the target data
train_test_data_mistral = data['target']

# Save the target column as a CSV file
train_test_data_mistral.to_csv("/content/train_test_data_mistral.csv", index=False)

In [ ]:
train_test_data_mistral = pd.read_csv("/content/train_test_data_mistral.csv")


In [ ]:
train_test_data_mistral.head(5)

,target
0,<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে...
1,<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে...
2,<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে...
3,<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে...
4,<s>### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে...


In [ ]:
from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd



# Split the dataset into train and test sets
train_data, test_data = train_test_split(train_test_data_mistral, test_size=0.1, random_state=42)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

 # Create a datasets.Dataset object
train = Dataset.from_dict(train_data)
test = Dataset.from_dict(test_data)
# Print the dataset to check the format
print("Train Dataset : ",train)
print("Test Dataset : ", test)


Train Dataset :  Dataset({
    features: ['target'],
    num_rows: 46584
})
Test Dataset :  Dataset({
    features: ['target'],
    num_rows: 5176
})


In [ ]:
# Create DatasetDict
datasets = DatasetDict({
    'train': train,
    'test': test,
})

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['target'],
        num_rows: 46584
    })
    test: Dataset({
        features: ['target'],
        num_rows: 5176
    })
})

In [ ]:
datasets['train']

Dataset({
    features: ['target'],
    num_rows: 46584
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# !pip install accelerate bitsandbytes

In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
prompt="### Instruction:\nপ্রদত্ত ইনপুট ব্যবহার করে একটি নির্দেশ তৈরি করুন যা এলএলএম দ্বারা প্রতিক্রিয়া উত্পাদনের জন্য ব্যবহৃত হতে পারে.\n\n### input:\nসুস্থ থাকার তিনটি উপায় বলুন।\n\n### output:"

In [ ]:
generate_response(prompt, model)

'<s> \n1. প্रসাদ\n2. সুস্থ\n3. ঘস এল់\n\n### Note:\n- ঘস এলිन থেন মুढ़य और और और और और और और और और और खा एले और और और और और और एले हैं।\n- चीकुर इन उपाয় বोलें।\n- और और और और और उपाय बोलें।\n- और और और और और उपाযय बोलें।\n- और और और और और और उपाய बोलें।\n- और और और और और उपाى बोलें।\n- और और और और और और और और उपाয় बोलें।\n- और और और और और और और और और उपाय बोलें।\n- और और और और और और और और और और एलती और और और और और एलती हैं।\n- और और और और और और और और और और एलती हैं।\n- और और और और और और और और और और और और और और उपाय बोलें।\n- और और और और और और और और और देउर और और और और और और और और और और ताएं।\n- और और और और और और और और और और और और और वाएं।\n- और और और और और और और और और और एलती और और और और और एलती हैं।\n- और और और और और �'

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=False,
  lr_scheduler_type='constant',
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 100

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=format_data, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=datasets["train"],
  eval_dataset=datasets["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

ValueError: Error occurred while packing the dataset. Make sure that your dataset has enough samples to at least yield one packed sequence.